Import Model

In [1]:
# from transformers import pipeline
# classifier = pipeline("text-classification", model="Hate-speech-CNERG/bert-base-uncased-hatexplain", device=0)

E:\scientificProjectXAI\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [1]:
import json
import numpy as np

# with open("../SHAP_values/combined_shap_values.json", "r") as f:
#     shap_data = json.load(f)

with open("../data/annotated_labels.json", "r") as f:
    annotated_labels = json.load(f)

with open("../data/classified_labels.json", "r") as f:
    classified_labels = json.load(f)

with open('../data/original_post_tokens.json', 'r', encoding='utf-8') as f:
    original_post_tokens = json.load(f)

with open('../data/original_sentences.json', 'r', encoding='utf-8') as f:
    original_sentences = json.load(f)

with open('../data/rationale_tokens_list.json', 'r') as f:
    rationale_tokens_data_full = json.load(f)

with open('../data/classified_results.json', 'r') as f:
    classified_results = json.load(f)

Extract original data from JSON file

In [3]:
# import json
# 
# json_file = '../data/dataset.json'
# 
# def extract_data(file):
#     with open(file, 'r', encoding='utf-8') as f:
#         data = json.load(f)
# 
#     sentences = []
#     abuse_flags = []
#     
#     for key, entry in data.items():
#         if 'post_tokens' in entry:
#             post_tokens = entry['post_tokens']
#             sentence = " ".join(post_tokens)
#         else:
#             sentence = " "
#             print(f"Warning: Entry {key} is missing 'post_tokens' key")
#         
#         if 'annotators' in entry:
#             labels = [annotator['label'] for annotator in entry['annotators']]
#             if sum(label != "normal" for label in labels) >= 2:
#                 abuse_label = 0  # Abusive
#             else:
#                 abuse_label = 1  # normal
#         else:
#             abuse_label = 0  # Default to normal if 'annotators' key is missing
#             print(f"Warning: Entry {key} is missing 'annotators' key")
#         
#         sentences.append(sentence)
#         abuse_flags.append(abuse_label)
# 
#     return sentences, abuse_flags
# 
# original_sentences, annotated_labels = extract_data(json_file) #length = 20148
# 
# with open("../data/original_sentences.json", "w") as f:
#     json.dump(original_sentences, f, indent=4)

Generate the messages for GPT Assistant messages

1. Do the predictions using the classify model
2. Reshape the classified results
3. Store the shaped messages

In [14]:
import numpy
# classified_results = classifier(original_sentences, return_all_scores=True) #top 10 instances
# print(original_sentences[0])

In [23]:
# print(classified_results)
# 
# for instance in classified_results:
#     print(instance)
    
# classified_results_string_list = [', '.join([str(element) for element in instance]) for instance in classified_results]
# for i in original_sentences[:3]:
#     print(i)
# with open("../data/classified_results.json", "w") as f:
#     json.dump(classified_results_string_list, f, indent=4)
# for s in classified_results_string_list[:5]:
#     print(s)
# {'label': 'hate speech', 'score': 0.03392261639237404}, {'label': 'normal', 'score': 0.770871102809906}, {'label': 'offensive', 'score': 0.19520631432533264}
# {'label': 'hate speech', 'score': 0.07430059462785721}, {'label': 'normal', 'score': 0.6370931267738342}, {'label': 'offensive', 'score': 0.2886062264442444}


In [2]:
def results_convert_to_gpt_prompt_statement(sentences, classified_results_with_probabilities):
    # if len(sentences) != len(classified_results_with_probabilities):
    #     raise ValueError("The length of original_sentences and classified_results_string_list must be the same.")
    
    prompt_statements = []
    
    for sentence, result in zip(sentences, classified_results_with_probabilities):
        combined_string = f'Original text: "{sentence}", The classification result (probability distribution) of the prediction model for this text: {result}'
        prompt_statements.append(combined_string)
    
    return prompt_statements

messages_prompted_to_GPT = results_convert_to_gpt_prompt_statement(original_sentences, classified_results)

# for i in messages_GPT_inputs[1567:1568]:
#     print(i)
# Original text: "i dont think im getting my baby them white 9 he has two white j and nikes not even touched", Probability distribution of prediction model output: {'label': 'hate speech', 'score': 0.03392261639237404}, {'label': 'normal', 'score': 0.770871102809906}, {'label': 'offensive', 'score': 0.19520631432533264}

GPT Assistant API

In [7]:
# from openai import OpenAI
# 
# client = OpenAI()
# 
# thread = client.beta.threads.create()
# 
# message = client.beta.threads.messages.create(
#   thread_id=thread.id,
#   role="user",
#   content=messages_prompted_to_GPT[4]
# )


# run = client.beta.threads.create_and_run(
#   assistant_id="asst_JFDjSdtxCjJq7pVGm8YGxKrZ",
#   thread={
#     "messages": [
#       {"role": "user", "content": messages_GPT_inputs[4]}
#     ]
#   }
# )
 
# print(run)

# completion = client.chat.completions.create(
#     model="gpt-4o-mini",
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {
#             "role": "user",
#             "content": "Write a haiku about recursion in programming."
#         }
#     ]
# )
# 
# print(completion.choices[0].message)

GPT Assistant with Streaming

In [8]:
# from typing_extensions import override
# from openai import AssistantEventHandler
#  
# # First, we create a EventHandler class to define
# # how we want to handle the events in the response stream.
#  
# class EventHandler(AssistantEventHandler):    
#   @override
#   def on_text_created(self, text) -> None:
#     print(f"\nassistant > ", end="", flush=True)
#       
#   @override
#   def on_text_delta(self, delta, snapshot):
#     print(delta.value, end="", flush=True)
#       
#   def on_tool_call_created(self, tool_call):
#     print(f"\nassistant > {tool_call.type}\n", flush=True)
#   
#   def on_tool_call_delta(self, delta, snapshot):
#     if delta.type == 'code_interpreter':
#       if delta.code_interpreter.input:
#         print(delta.code_interpreter.input, end="", flush=True)
#       if delta.code_interpreter.outputs:
#         print(f"\n\noutput >", flush=True)
#         for output in delta.code_interpreter.outputs:
#           if output.type == "logs":
#             print(f"\n{output.logs}", flush=True)
#  
# # Then, we use the `stream` SDK helper 
# # with the `EventHandler` class to create the Run 
# # and stream the response.
#  
# with client.beta.threads.runs.stream(
#   thread_id=thread.id,
#   assistant_id='asst_JFDjSdtxCjJq7pVGm8YGxKrZ',
#   event_handler=EventHandler(),
# ) as stream:
#   stream.until_done()

GPT Assistant without Streaming

In [9]:
# run = client.beta.threads.runs.create_and_poll(
#   thread_id=thread.id,
#   assistant_id='asst_JFDjSdtxCjJq7pVGm8YGxKrZ'
#   # response_format='json_object'
# )

In [10]:
# if run.status == 'completed': 
# messages = client.beta.threads.messages.retrieve(
#     thread_id=thread.id,
#     message_id=message.id
# )
# print(messages)
# else:
#   print(run.status)

# messages = client.beta.threads.messages.list(
#     thread_id=thread.id
# )
# 
# response = messages.data[0].content[0].text.value
# 
# print(response)

This text is classified as Offensive.

The following words may have inappropriate meanings: ***bitch, hate, white, bitches***

Explanation: The possible explanation is that the words/usage ***bitch*** and ***bitches*** are derogatory terms used to demean individuals, particularly women, while ***hate*** expresses strong negative feelings towards a group, and ***white*** is used in a derogatory context indicating a racial element. These usages contribute to the classification of the text as offensive.


Loop to send prompt statements to GPT to get reply

In [12]:
from openai import OpenAI

client = OpenAI()

# thread_messages_responses = []
run_ids = []

thread = client.beta.threads.create()

for message_content in messages_prompted_to_GPT[:2000]:
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=message_content
    )

    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id='asst_JFDjSdtxCjJq7pVGm8YGxKrZ'
    )
    
    run_ids.append(run.id)

    # messages = client.beta.threads.messages.list(
    #     thread_id=thread.id
    # )

    # if run.status == 'completed':
    #     messages = client.beta.threads.messages.list(
    #         thread_id=thread.id
    #     )
        # print(messages)
    # else:
        # print(run.status)

        # response = messages.data[0].content[0].text.value
        # responses.append(response)

BadRequestError: Error code: 400 - {'error': {'message': 'Thread thread_XSPoNyv5dfB5Z6LN0OFwfKCb already has an active run run_LvQEUk4iNmPtAj5BFf3WbxBu.', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [7]:
actual_response_texts = []
# run_ids.append("59598")
for run_id in run_ids:
    thread_message = client.beta.threads.messages.list(
        thread_id=thread.id,
        run_id=run_id
    )
    # print(thread_message, "\n")
    if not thread_message.data:
        # print("NONE for this")
        actual_response_text = ""
    else:
        actual_response_text = thread_message.data[0].content[0].text.value
        
    actual_response_texts.append(actual_response_text)
# print(len(thread_messages_responses))
# for j, i in enumerate(actual_response_texts):
#     print(j, i, "\n")


In [72]:
# for i in actual_response_texts:
#     print(i)

This text is classified as normal.

The following words may have inappropriate meanings: ***baby, white, touched***

Explanation: The possible explanation is that the word/usage ***baby*** can sometimes refer to a person in a derogatory manner, while ***white*** may have racial connotations depending on the context, but here it seems to refer to the color of shoes. The word ***touched*** could imply unwanted physical contact in other contexts, but in this case, it appears to refer to the condition of the shoes.
This text is classified as normal.

The following words may have inappropriate meanings: ***womxn, offences, queer***

Explanation: The possible explanation is that the term ***womxn*** is used as an inclusive alternative to "women," which some may find controversial or inappropriate in certain contexts. The word ***offences*** refers to legal violations, which can be sensitive but is used in a factual manner here. The term ***queer*** can be considered offensive in some context

In [38]:
# for i in responses:
#     print(i)

This text is classified as normal.

The following words may have inappropriate meanings: ***baby, white, j, nikes***

Explanation: The possible explanation is that the word/usage ***baby*** refers to a child or loved one, which is generally neutral, while ***white*** in this context is likely referring to the color of shoes and does not carry an inappropriate meaning. The terms ***j*** and ***nikes*** refer to specific sneaker models, which are also neutral in this context.
This text is classified as normal.

The following words may have inappropriate meanings: ***womxn, trans, lesbian, bisexual, queer***

Explanation: The possible explanation is that the terms ***womxn***, ***trans***, ***lesbian***, ***bisexual***, and ***queer*** are used to describe various identities and orientations within the LGBTQ+ community. In this context, they are used to advocate for inclusivity and do not carry inappropriate meanings.
This text is classified as hate speech.

The following words may have i

In [9]:
import re

important_words_GPT = []
def extract_inappropriate_words(text):
    pattern = r"\*\*\*(.*?)\*\*\*"
    matches = re.findall(pattern, text)
    
    if matches:
        words_list = [word.strip() for word in matches[0].split(',')]
        return words_list
    return []

for actual_response_text in actual_response_texts:
    # print(type(actual_response_text))
    important_words = extract_inappropriate_words(actual_response_text)
    important_words_GPT.append(important_words)
    
for i in important_words_GPT:
    print(i)

# print(len(important_words_GPT))

['baby', 'white', 'touched']
['womxn', 'offences', 'sexual']
['nawt', 'niggers', 'ignoring']
['chinese', 'ching chong', 'pakistani']
['bitch', 'hate', 'white']
['raped', 'kike', 'exterminate', 'goyim']
['nigger', 'trainee', 'scary']
['nogs', 'jews', 'dykes']
['guilty', 'jew', 'nigger', 'lover']
['abortion', 'niglets', 'rapists', 'welfare', 'democrat']
['multiculturalism', 'muzzie', 'minaret']
['niggas']
['gay', 'chemicals']
['closet', 'gay', 'obsessed']
['hoes', 'thots']
['niggas']
['niggas']
['nazi', 'white supremacist']
['negress', 'white', 'black', 'love', 'nobama']
['kike']
['israel']
['kps', 'islam', 'terrorist', 'butured']
['islamophobia', 'white supremacy']
['kike']
['nigger', 'nazis']
['beaner']
['albino', 'nigger']
['spic']
['kike']
['spic']
['moslem', 'welfare', 'taxpayer', 'homeless vets']
['negro', 'moslem']
['dyke']
['sumbitch']
['bitch']
['black']
['bitch']
['bitches', 'niggas']
['queer', 'trans']
['white folks']
['faggots']
['blacks', 'jews', 'white']
['invaders']
['sche

In [11]:
spanr_count = 0
spanr_total = 0
cover_total = 0
cover_correct = 0

rationale_tokens_data = rationale_tokens_data_full[:100]

for i in range(100):
    if annotated_labels[i] == 1:
        rationale_tokens = set(rationale_tokens_data[i])
        # important_tokens = set(important_words_data[i]["important_words"])
        important_tokens = set(important_words_GPT[i])

        spanr_total += 1
        if rationale_tokens & important_tokens:
            # print(i, rationale_tokens & important_tokens)
            spanr_count += 1
            cover_correct += len(rationale_tokens & important_tokens)
        cover_total += len(rationale_tokens)

if cover_total > 0:
    cover = cover_correct / cover_total
else:
    cover = 0

spanr = spanr_count / spanr_total

print(f"SpanR: {spanr}")
print(f"Cover: {cover}")
print(spanr_total, spanr_count)

SpanR: 0.8478260869565217
Cover: 0.3155339805825243
46 39
